In [2]:
!git clone https://github.com/UL-FRI-NLP-Course/ul-fri-nlp-course-project-2024-2025-drop-table-teams

Cloning into 'ul-fri-nlp-course-project-2024-2025-drop-table-teams'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 193 (delta 7), reused 8 (delta 6), pack-reused 175 (from 4)
Receiving objects: 100% (193/193), 1.31 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [3]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "ul-fri-nlp-course-project-2024-2025-drop-table-teams"))

In [4]:
#!pip install -r "ul-fri-nlp-course-project-2024-2025-drop-table-teams/requirements.txt"
!pip install langchain==0.3.24 langchain_community==0.3.22 keybert==0.9.0 faiss-cpu==1.10.0 bitsandbytes==0.45.5
!pip install python-docx==1.1.2 pypdf==5.4.0 unstructured==0.17.2 scholarly==1.7.11 requests==2.32.3 pymupdf
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 811.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

In [14]:
import torch
import os

from sentence_transformers import CrossEncoder
from src.chain_initialisation import init_chain
from src.embedding_management import init_embeddings, add_embeddings_from_files
from src.model_initialisation import init_model
from src.pipeline_initialisation import init_pipeline
from src.retriever_initialisation import init_retriever
from src.dynamic_doc_retrieval import download_documents, initialise_keyword_model, generate_query_from_question
from src.dataset_scraping import scrap_dataset

In [15]:
scrap_dataset()

Extracted Krapivin2009 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/Krapivin2009
Extracted Nguyen2007 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/Nguyen2007
Extracted PubMed to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed
Extracted Schutz2008 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/Schutz2008
Extracted SemEval2010 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/SemEval2010
Extracted SemEval2017 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/SemEval2017
Extracted citeulike180 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/citeulike180
Extracted fao30 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao30
Extracted fao780 to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/fao780
Extracted kdd to /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/kdd
Extracte

In [27]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
df

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...
...,...,...,...,...,...,...
11674,The enzyme pepsin plays an important role in t...,lipids,protons,proteins,peptides,Protein A large part of protein digestion take...
11675,What remains a constant of radioactive substan...,acidity,temperature,volatility,rate of decay,The rate of decay of a radioactive substance i...
11676,"Terrestrial ecosystems, also known for their d...",substrates,bisomes,monomes,biomes,"Terrestrial ecosystems, also known for their d..."
11677,High explosives create shock waves that exceed...,turbulence,light speed,ion speed,supersonic,The modern day formulation of gun powder is ca...


In [7]:
BASE_DIR = os.curdir
DATA_DIR = os.path.join(BASE_DIR, "data")
DYNAMIC_DATA_DIR = os.path.join(DATA_DIR, "dynamic")
print(f"dynamic data dir: {DYNAMIC_DATA_DIR}")


def rerank_documents(reranker, query: str, documents: list, top_k: int = 3, min_score = 0.5) -> list:
    if not documents:
        return []

    pairs = [(query, doc.page_content) for doc in documents]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, score in ranked[:top_k] if score > min_score]

def ask_question(model_chain, reranker, keyword_model, vectorstore, question, num_docs, max_tries, online, rerank_top_k, rerank_min_score):
    if online:
        print("Obtaining data from scholar...")
        query = generate_query_from_question(keyword_model, question)
        downloaded_files = download_documents(query, DYNAMIC_DATA_DIR, num_docs, max_tries)
        add_embeddings_from_files(vectorstore, downloaded_files)

    response = model_chain.invoke({
        "question": question,
        "chat_history": []
    })
    answer = response["answer"].split("### Answer:")[-1].strip()
    source_documents = response["source_documents"]
    filtered_docs = rerank_documents(reranker, question, source_documents, rerank_top_k, rerank_min_score)
    return answer, source_documents, filtered_docs

dynamic data dir: ./data/dynamic


In [8]:
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz

# Initialize embedding model
evaluation_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
os.environ["hf_token"] = YOUR_TOKEN
os.environ["llm_name"] = "mistralai/Mistral-7B-Instruct-v0.2"
os.environ["embedding_model"] = "sentence-transformers/all-mpnet-base-v2"
os.environ["vectorstore_k"] = "5"
os.environ["similarity_threshold"] = "0.8"
os.environ["vectorstore_document_limit"] = "100"
os.environ["num_docs"] = "3"
os.environ["max_tries"] = "10"
os.environ["use_Scholar"] = "True"

import subprocess
subprocess.run(["huggingface-cli", "login", "--token", YOUR_TOKEN])

In [18]:
cuda_available = torch.cuda.is_available()
print(f"Initializing model... CUDA available: {cuda_available}")
model, model_name = init_model(cuda_available)
pipeline = init_pipeline(model=model, model_name=model_name)
vectorstore = init_embeddings(cuda_available)
retriever = init_retriever(vectorstore)
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
chain = init_chain(pipeline=pipeline, retriever=retriever)
keyword_model = initialise_keyword_model()
print("Start-up complete.")

Initializing model... CUDA available: True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


FAISS index not found. Building...
Error reading /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed/PubMed/docs_keys/12419768.txt: 'utf-8' codec can't decode byte 0xfc in position 57: invalid start byte
Error reading /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed/PubMed/docs_keys/15353412.txt: 'utf-8' codec can't decode byte 0xb0 in position 10298: invalid start byte
Error reading /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed/PubMed/docs_keys/12777364.txt: 'utf-8' codec can't decode byte 0xe1 in position 105: invalid continuation byte
Error reading /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed/PubMed/docs_keys/10933267.txt: 'utf-8' codec can't decode byte 0xdf in position 1781: invalid continuation byte
Error reading /content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/data/PubMed/PubMed/docs_keys/16269586.txt: 'utf-8' codec can't decode byte 0xf8 in position 88: invalid s

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

/content/ul-fri-nlp-course-project-2024-2025-drop-table-teams/src/chain_initialisation.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


LLM model chain created.
Start-up complete.


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
drive_path = "/content/drive/MyDrive/Colab Notebooks/"


In [29]:
def evaluate_answer(row, chain, reranker, keyword_model, vectorstore, num_docs, max_tries, online, rerank_top_k, rerank_min_score):
    question = row['question']
    correct_answer = row['correct_answer']
    support = row['support']
    distractors = [row['distractor1'], row['distractor2'], row['distractor3']]

    # Chatbot answer
    response, source_docs, filtered_docs = ask_question(chain, reranker, keyword_model, vectorstore, question, num_docs, max_tries, online, rerank_top_k, rerank_min_score)
    response = response.lower()

    # Exact match
    exact = int(correct_answer.lower() in response)

    # Fuzzy match
    fuzzy_score = fuzz.partial_ratio(correct_answer.lower(), response)

    # Embedding similarity to correct answer
    emb_correct = evaluation_model.encode(correct_answer, convert_to_tensor=True)
    emb_response = evaluation_model.encode(response, convert_to_tensor=True)
    sim_correct = util.cos_sim(emb_correct, emb_response).item()

    # Embedding similarity to support passage
    emb_support = evaluation_model.encode(support, convert_to_tensor=True)
    sim_support = util.cos_sim(emb_support, emb_response).item()

    # Compare similarity to distractors
    sim_distractors = []
    for d in distractors:
        emb_distractor = evaluation_model.encode(d, convert_to_tensor=True)
        sim_distractors.append(util.cos_sim(emb_distractor, emb_response).item())

    # Is chatbot closer to correct than to all distractors?
    correct_beats_distractors = int(sim_correct > max(sim_distractors))

    return {
        "question": question,
        "chatbot_response": response,
        "exact_match": exact,
        "fuzzy_score": fuzzy_score,
        "sim_to_correct": round(sim_correct, 3),
        "sim_to_support": round(sim_support, 3),
        "correct_beats_distractors": correct_beats_distractors,
        "source_docs": source_docs,
        "filtered_docs": filtered_docs
    }

In [ ]:

configs = [
    {"k": 3, "th": 0.7},
    {"k": 5, "th": 0.7},
    {"k": 5, "th": 0.85},
    {"k": 10, "th": 0.9},
]

rerank_configs = [
    {"top_k": 2, "min_score": 0.3},
    {"top_k": 3, "min_score": 0.5},
    {"top_k": 5, "min_score": 0.6},
]


for config in configs:
    retriever = init_retriever(vectorstore, k=config["k"], th=config["th"])
    reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    chain = init_chain(pipeline=pipeline, retriever=retriever)
    keyword_model = initialise_keyword_model()
    num_docs: int = int(os.getenv("num_docs"))
    max_tries: int = int(os.getenv("max_tries"))
    print("Start-up complete.")
    for online in [True, False]:
        for rerank_conf in rerank_configs:
            results = []
            filename = f"k_{config['k']}-thresh_{config['th']}-online_{online}-top_k_{rerank_conf['top_k']}-min_score_{rerank_conf['min_score']}.csv"
            for i, row in df[:20].iterrows():
                result = evaluate_answer(row, chain, reranker, keyword_model, vectorstore, num_docs, max_tries, online, rerank_conf["top_k"], rerank_conf["min_score"])
                results.append(result)

            results_df = pd.DataFrame(results)
            results_df.to_csv(f"{drive_path}/{filename}", index=False)

Retriever initialized with k=3 and similarity threshold=0.7
LLM model chain created.
Start-up complete.
Obtaining data from scholar...
Querying Google Scholar for: yogurt foods cheese organism preparation
